## Atividade Titanic


In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, accuracy_score

import time

from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from autosklearn.classification import AutoSklearnClassifier

/home/bruno/.local/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


# Explorando os dados

In [2]:
gender = pd.read_csv('data/gender_submission.csv', index_col=False, squeeze=True)
gender.dropna()

PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]

In [3]:
train = pd.read_csv('data/train.csv', index_col=False, squeeze=True)

#realiza leitura e já remove linhas com valores NaN
train["Age"] = train["Age"].fillna(train["Age"].mean())
train

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex        Age  \
0                              Braund, Mr. Owen Harris    male  22.000000   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.000000   
2                               Heikkinen, Miss. Laina  female  26.000000   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.000000   
4                             Allen, Mr. William Henry    male  35.000000   
..                                                 ...     ...        ...   
886                              Montvila, Rev. Juozas    male  27.000000   
887                       Graham, Miss. Margaret Edith  female  19.000000   
888           Johnston, Miss. Catherine Helen "Carrie"  female  29.699118   
889                              Behr, Mr. Karl Howell    male  26.000000   
890                                Dooley, Mr. Patrick    male  32.000000   

     SibSp  Parch            Ticket     Fare Cabin Embarked  
0        1      0         A/5 21171   7.2500   NaN        S  
1        1      0          PC 17599  71.2833   C85        C  
2        0      0  STON/O2. 3101282   7.9250   NaN        S  
3        1      0            113803  53.1000  C123        S  
4        0      0            373450   8.0500   NaN        S  
..     ...    ...               ...      ...   ...      ...  
886      0      0            211536  13.0000   NaN        S  
887      0      0            112053  30.0000   B42        S  
888      1      2        W./C. 6607  23.4500   NaN        S  
889      0      0            111369  30.0000  C148        C  
890      0      0            370376   7.7500   NaN        Q  

[891 rows x 12 columns]

In [4]:
#printando então a matriz de correlação, notamos que o atributo "Fare" tem uma boa correlação com
#os membros sobreviventes, vamos explorar melhor
train[["PassengerId", "Survived", "Pclass", "Age", 'SibSp', "Parch", "Fare"]].corr()

PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.035144  0.033207 -0.057527 -0.001652   
Survived       -0.005007  1.000000 -0.338481 -0.069809 -0.035322  0.081629   
Pclass         -0.035144 -0.338481  1.000000 -0.331339  0.083081  0.018443   
Age             0.033207 -0.069809 -0.331339  1.000000 -0.232625 -0.179191   
SibSp          -0.057527 -0.035322  0.083081 -0.232625  1.000000  0.414838   
Parch          -0.001652  0.081629  0.018443 -0.179191  0.414838  1.000000   
Fare            0.012658  0.257307 -0.549500  0.091566  0.159651  0.216225   

                 Fare  
PassengerId  0.012658  
Survived     0.257307  
Pclass      -0.549500  
Age          0.091566  
SibSp        0.159651  
Parch        0.216225  
Fare         1.000000

In [5]:
def toNumerical(df, attr):
    return pd.factorize(df[attr])[0]

#converte alguns atributos categóricos para numéricos para análise de correlação
train['Sex'] = toNumerical(train, 'Sex')
train['Embarked'] = toNumerical(train, 'Embarked')
train

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex        Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.000000      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.000000      1   
2                               Heikkinen, Miss. Laina    1  26.000000      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.000000      1   
4                             Allen, Mr. William Henry    0  35.000000      0   
..                                                 ...  ...        ...    ...   
886                              Montvila, Rev. Juozas    0  27.000000      0   
887                       Graham, Miss. Margaret Edith    1  19.000000      0   
888           Johnston, Miss. Catherine Helen "Carrie"    1  29.699118      1   
889                              Behr, Mr. Karl Howell    0  26.000000      0   
890                                Dooley, Mr. Patrick    0  32.000000      0   

     Parch            Ticket     Fare Cabin  Embarked  
0        0         A/5 21171   7.2500   NaN         0  
1        0          PC 17599  71.2833   C85         1  
2        0  STON/O2. 3101282   7.9250   NaN         0  
3        0            113803  53.1000  C123         0  
4        0            373450   8.0500   NaN         0  
..     ...               ...      ...   ...       ...  
886      0            211536  13.0000   NaN         0  
887      0            112053  30.0000   B42         0  
888      2        W./C. 6607  23.4500   NaN         0  
889      0            111369  30.0000  C148         1  
890      0            370376   7.7500   NaN         2  

[891 rows x 12 columns]

In [6]:
#podemos notar claramente que o sexo é um fator que pode ser bastante útil para identificar a sobrevivência
#isso, olhando para as correlações. Vamos tentar identificar outras informações do dataset
train[["PassengerId", "Survived", "Age", "Pclass", 'SibSp', "Fare", "Sex", "Embarked"]].corr()

PassengerId  Survived       Age    Pclass     SibSp      Fare  \
PassengerId     1.000000 -0.005007  0.033207 -0.035144 -0.057527  0.012658   
Survived       -0.005007  1.000000 -0.069809 -0.338481 -0.035322  0.257307   
Age             0.033207 -0.069809  1.000000 -0.331339 -0.232625  0.091566   
Pclass         -0.035144 -0.338481 -0.331339  1.000000  0.083081 -0.549500   
SibSp          -0.057527 -0.035322 -0.232625  0.083081  1.000000  0.159651   
Fare            0.012658  0.257307  0.091566 -0.549500  0.159651  1.000000   
Sex            -0.042939  0.543351 -0.084153 -0.131900  0.114631  0.182333   
Embarked       -0.030323  0.101849  0.001932  0.050992 -0.058008  0.058462   

                  Sex  Embarked  
PassengerId -0.042939 -0.030323  
Survived     0.543351  0.101849  
Age         -0.084153  0.001932  
Pclass      -0.131900  0.050992  
SibSp        0.114631 -0.058008  
Fare         0.182333  0.058462  
Sex          1.000000  0.111249  
Embarked     0.111249  1.000000

In [7]:
#podemos notar então, que no dataset, temos 549 mortos e 342 sobreviventes
train.groupby("Survived")["Survived"].count()

Survived
0    549
1    342
Name: Survived, dtype: int64

In [8]:
#olhando agora para o atributo Passagem
train.describe()

PassengerId    Survived      Pclass         Sex         Age  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642    0.352413   29.699118   
std     257.353842    0.486592    0.836071    0.477990   13.002015   
min       1.000000    0.000000    1.000000    0.000000    0.420000   
25%     223.500000    0.000000    2.000000    0.000000   22.000000   
50%     446.000000    0.000000    3.000000    0.000000   29.699118   
75%     668.500000    1.000000    3.000000    1.000000   35.000000   
max     891.000000    1.000000    3.000000    1.000000   80.000000   

            SibSp       Parch        Fare    Embarked  
count  891.000000  891.000000  891.000000  891.000000  
mean     0.523008    0.381594   32.204208    0.359147  
std      1.102743    0.806057   49.693429    0.638707  
min      0.000000    0.000000    0.000000   -1.000000  
25%      0.000000    0.000000    7.910400    0.000000  
50%      0.000000    0.000000   14.454200    0.000000  
75%      1.000000    0.000000   31.000000    1.000000  
max      8.000000    6.000000  512.329200    2.000000

In [9]:
#Como podemos perceber, pela média e desvio padrão, podemos dividir os atributos em grupos de 
#aproximadamente 70 em 70. Fazendo isso:

def agroupFare(row):
    fare = row["Fare"]
    if(fare < 70): 
        return 0
    if(fare < 140):
        return 1
    if(fare < 210):
        return 2
    if(fare < 280):
        return 3
    if(fare < 350):
        return 4
    if(fare < 420):
        return 5
    if(fare < 500):
        return 6
    return 7

train["FareGroup"] = train.apply (lambda row: agroupFare(row), axis=1)
train

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex        Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.000000      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.000000      1   
2                               Heikkinen, Miss. Laina    1  26.000000      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.000000      1   
4                             Allen, Mr. William Henry    0  35.000000      0   
..                                                 ...  ...        ...    ...   
886                              Montvila, Rev. Juozas    0  27.000000      0   
887                       Graham, Miss. Margaret Edith    1  19.000000      0   
888           Johnston, Miss. Catherine Helen "Carrie"    1  29.699118      1   
889                              Behr, Mr. Karl Howell    0  26.000000      0   
890                                Dooley, Mr. Patrick    0  32.000000      0   

     Parch            Ticket     Fare Cabin  Embarked  FareGroup  
0        0         A/5 21171   7.2500   NaN         0          0  
1        0          PC 17599  71.2833   C85         1          1  
2        0  STON/O2. 3101282   7.9250   NaN         0          0  
3        0            113803  53.1000  C123         0          0  
4        0            373450   8.0500   NaN         0          0  
..     ...               ...      ...   ...       ...        ...  
886      0            211536  13.0000   NaN         0          0  
887      0            112053  30.0000   B42         0          0  
888      2        W./C. 6607  23.4500   NaN         0          0  
889      0            111369  30.0000  C148         1          0  
890      0            370376   7.7500   NaN         2          0  

[891 rows x 13 columns]

In [10]:
#Cria também um novo atributo contendo a soma dos familiares
train['family'] = train["SibSp"] + train['Parch']
train

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex        Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.000000      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.000000      1   
2                               Heikkinen, Miss. Laina    1  26.000000      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.000000      1   
4                             Allen, Mr. William Henry    0  35.000000      0   
..                                                 ...  ...        ...    ...   
886                              Montvila, Rev. Juozas    0  27.000000      0   
887                       Graham, Miss. Margaret Edith    1  19.000000      0   
888           Johnston, Miss. Catherine Helen "Carrie"    1  29.699118      1   
889                              Behr, Mr. Karl Howell    0  26.000000      0   
890                                Dooley, Mr. Patrick    0  32.000000      0   

     Parch            Ticket     Fare Cabin  Embarked  FareGroup  family  
0        0         A/5 21171   7.2500   NaN         0          0       1  
1        0          PC 17599  71.2833   C85         1          1       1  
2        0  STON/O2. 3101282   7.9250   NaN         0          0       0  
3        0            113803  53.1000  C123         0          0       1  
4        0            373450   8.0500   NaN         0          0       0  
..     ...               ...      ...   ...       ...        ...     ...  
886      0            211536  13.0000   NaN         0          0       0  
887      0            112053  30.0000   B42         0          0       0  
888      2        W./C. 6607  23.4500   NaN         0          0       3  
889      0            111369  30.0000  C148         1          0       0  
890      0            370376   7.7500   NaN         2          0       0  

[891 rows x 14 columns]

# Seleção das Features

In [11]:
modelDataset = train[["PassengerId", "Survived", "Age", 'Pclass', "Fare", "FareGroup", "Sex", "Embarked", "family"]]
modelDataset.corr()

PassengerId  Survived       Age    Pclass      Fare  FareGroup  \
PassengerId     1.000000 -0.005007  0.033207 -0.035144  0.012658   0.014890   
Survived       -0.005007  1.000000 -0.069809 -0.338481  0.257307   0.208589   
Age             0.033207 -0.069809  1.000000 -0.331339  0.091566   0.081910   
Pclass         -0.035144 -0.338481 -0.331339  1.000000 -0.549500  -0.438254   
Fare            0.012658  0.257307  0.091566 -0.549500  1.000000   0.951401   
FareGroup       0.014890  0.208589  0.081910 -0.438254  0.951401   1.000000   
Sex            -0.042939  0.543351 -0.084153 -0.131900  0.182333   0.149118   
Embarked       -0.030323  0.101849  0.001932  0.050992  0.058462   0.079204   
family         -0.040143  0.016639 -0.248512  0.065997  0.217138   0.075075   

                  Sex  Embarked    family  
PassengerId -0.042939 -0.030323 -0.040143  
Survived     0.543351  0.101849  0.016639  
Age         -0.084153  0.001932 -0.248512  
Pclass      -0.131900  0.050992  0.065997  
Fare         0.182333  0.058462  0.217138  
FareGroup    0.149118  0.079204  0.075075  
Sex          1.000000  0.111249  0.200988  
Embarked     0.111249  1.000000 -0.077927  
family       0.200988 -0.077927  1.000000

In [12]:
def perceptronCall(X_train,X_test,Y_train,Y_test):    
    #Aplicando Perceptron
    print(" Perceptron ")
    ini = time.time() #Inicio do calculo do tempo

    perceptron = Perceptron(tol=1e-8, random_state=0, eta0=0.5, n_iter_no_change=10, l1_ratio=0.5, warm_start=True, max_iter=10000)
    perceptron.fit(X_train, Y_train)

    end = time.time()
    perceptron_time = end - ini # Fim do calculo do tempo

    print(classification_report(Y_test, perceptron.predict(X_test)))
    print()

    #Aplicando MLP
    print(" MLP ")
    ini = time.time() #Inicio do calculo do tempo

    mlp = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(30, 21), random_state=1)
    mlp.fit(X_train, Y_train)

    end = time.time()
    mlp_time = end - ini # Fim do calculo do tempo

    print(classification_report(Y_test, mlp.predict(X_test)))
    print()

    return perceptron, mlp

def automlCall(X_train,X_test,Y_train,Y_test):
    #Aplicando AutoML
    print(" AutoML ")
    ini = time.time() #Inicio do calculo do tempo

    # define search
    automl = AutoSklearnClassifier(time_left_for_this_task=2*60, per_run_time_limit=30, n_jobs=8)
    # perform the search
    automl.fit(X_train, Y_train)

    end = time.time()
    aut_time = end - ini # Fim do calculo do tempo

    print(automl.sprint_statistics())
    print(classification_report(Y_test, automl.predict(X_test)))
    print()

    return automl

# Modelos Preditivos

Como o Perceptron é um algoritmo muito simplista, que nem chega a se enquadrar entre redes neurais, foi utilizado também o MLP (Multi Layer Perceptron) que é uma melhoria sob o algoritmo, caracterizado pelo conceito do Perceptron, porém dividido em camadas. Com o Perceptron puro, fica muito complicado modificar os atributos e obter um resultado mais interessante que a primeira submissão.

In [13]:
#partindo então para um análise preditiva com a seleção de features mais relevantes
X_train,X_test,Y_train,Y_test=train_test_split(modelDataset[["Embarked", "Fare", "Sex", "Pclass"]],modelDataset['Survived'], test_size=0.27, random_state=0)

perceptron, mlp = perceptronCall(X_train,X_test,Y_train,Y_test)

 Perceptron 
              precision    recall  f1-score   support

           0       0.90      0.65      0.75       149
           1       0.61      0.88      0.72        92

    accuracy                           0.74       241
   macro avg       0.75      0.77      0.74       241
weighted avg       0.79      0.74      0.74       241


 MLP 
              precision    recall  f1-score   support

           0       0.84      0.81      0.83       149
           1       0.71      0.75      0.73        92

    accuracy                           0.79       241
   macro avg       0.78      0.78      0.78       241
weighted avg       0.79      0.79      0.79       241




In [14]:
test = pd.read_csv('data/test.csv', index_col=False, squeeze=True)

# test = test.dropna()
test["Fare"] = test["Fare"].fillna(0)
test['Sex'] = toNumerical(test, 'Sex')
test['Embarked'] = toNumerical(test, 'Embarked')
test['Survived'] = perceptron.predict(test[["Embarked", "Fare", "Sex", "Pclass"]])
test

PassengerId  Pclass                                          Name  Sex  \
0            892       3                              Kelly, Mr. James    0   
1            893       3              Wilkes, Mrs. James (Ellen Needs)    1   
2            894       2                     Myles, Mr. Thomas Francis    0   
3            895       3                              Wirz, Mr. Albert    0   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)    1   
..           ...     ...                                           ...  ...   
413         1305       3                            Spector, Mr. Woolf    0   
414         1306       1                  Oliva y Ocana, Dona. Fermina    1   
415         1307       3                  Saether, Mr. Simon Sivertsen    0   
416         1308       3                           Ware, Mr. Frederick    0   
417         1309       3                      Peter, Master. Michael J    0   

      Age  SibSp  Parch              Ticket      Fare Cabin  Embarked  \
0    34.5      0      0              330911    7.8292   NaN         0   
1    47.0      1      0              363272    7.0000   NaN         1   
2    62.0      0      0              240276    9.6875   NaN         0   
3    27.0      0      0              315154    8.6625   NaN         1   
4    22.0      1      1             3101298   12.2875   NaN         1   
..    ...    ...    ...                 ...       ...   ...       ...   
413   NaN      0      0           A.5. 3236    8.0500   NaN         1   
414  39.0      0      0            PC 17758  108.9000  C105         2   
415  38.5      0      0  SOTON/O.Q. 3101262    7.2500   NaN         1   
416   NaN      0      0              359309    8.0500   NaN         1   
417   NaN      1      1                2668   22.3583   NaN         2   

     Survived  
0           0  
1           1  
2           0  
3           0  
4           1  
..        ...  
413         0  
414         1  
415         0  
416         0  
417         0  

[418 rows x 12 columns]

In [15]:
#testando agora com um algoritmo de auto ML
automl = automlCall(X_train,X_test,Y_train,Y_test)

 AutoML 
auto-sklearn results:
  Dataset name: b7e10c68-9d9c-11eb-a388-f5d25f5781f2
  Metric: accuracy
  Best validation score: 0.823256
  Number of target algorithm runs: 56
  Number of successful target algorithm runs: 50
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

              precision    recall  f1-score   support

           0       0.83      0.86      0.84       149
           1       0.76      0.71      0.73        92

    accuracy                           0.80       241
   macro avg       0.79      0.78      0.79       241
weighted avg       0.80      0.80      0.80       241




## Validação Cruzada

In [16]:
#Agora, vamos utilizar validação cruzada nos algoritmos para verificarmos sua acurácia
def cross_validation(model, x, y, n_folds=3):
    
    kf = StratifiedKFold(n_splits=n_folds)
    
    results_accuracy = []
    
    for train, test in kf.split(x, y):
        data_train, target_train = x[train], y[train]
        data_test, target_test   = x[test], y[test]
        
        model = model.fit(data_train, target_train)
        out = model.predict(data_test)
        
        accuracy = accuracy_score(target_test, out)
        results_accuracy.append(accuracy)
        
    print('accuracy')
    print(np.average(results_accuracy))

In [17]:
x = np.array(modelDataset[["Embarked", "Fare", "Sex", "Pclass"]].values)
y = np.array(modelDataset['Survived'].values)

print("Perceptron")
cross_validation(perceptron, x, y)

print("MLP")
cross_validation(mlp, x, y)

print("AutoML")
cross_validation(automl, x, y)

Perceptron
accuracy
0.7171717171717171
MLP
accuracy
0.7789001122334457
AutoML
/home/bruno/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
accuracy
0.7912457912457912


In [18]:
test = pd.read_csv('data/test.csv', index_col=False, squeeze=True)

# test = test.dropna()
test["Fare"] = test["Fare"].fillna(0)
test['Sex'] = toNumerical(test, 'Sex')
test['Embarked'] = toNumerical(test, 'Embarked')
test['Survived'] = automl.predict(test[["Embarked", "Fare", "Sex", "Pclass"]])
test

PassengerId  Pclass                                          Name  Sex  \
0            892       3                              Kelly, Mr. James    0   
1            893       3              Wilkes, Mrs. James (Ellen Needs)    1   
2            894       2                     Myles, Mr. Thomas Francis    0   
3            895       3                              Wirz, Mr. Albert    0   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)    1   
..           ...     ...                                           ...  ...   
413         1305       3                            Spector, Mr. Woolf    0   
414         1306       1                  Oliva y Ocana, Dona. Fermina    1   
415         1307       3                  Saether, Mr. Simon Sivertsen    0   
416         1308       3                           Ware, Mr. Frederick    0   
417         1309       3                      Peter, Master. Michael J    0   

      Age  SibSp  Parch              Ticket      Fare Cabin  Embarked  \
0    34.5      0      0              330911    7.8292   NaN         0   
1    47.0      1      0              363272    7.0000   NaN         1   
2    62.0      0      0              240276    9.6875   NaN         0   
3    27.0      0      0              315154    8.6625   NaN         1   
4    22.0      1      1             3101298   12.2875   NaN         1   
..    ...    ...    ...                 ...       ...   ...       ...   
413   NaN      0      0           A.5. 3236    8.0500   NaN         1   
414  39.0      0      0            PC 17758  108.9000  C105         2   
415  38.5      0      0  SOTON/O.Q. 3101262    7.2500   NaN         1   
416   NaN      0      0              359309    8.0500   NaN         1   
417   NaN      1      1                2668   22.3583   NaN         2   

     Survived  
0           0  
1           1  
2           0  
3           0  
4           1  
..        ...  
413         0  
414         1  
415         0  
416         0  
417         0  

[418 rows x 12 columns]

Como podemos perceber, por meio de validação cruzada, o autoML se mostrou mais efetivo. Porém, bem mais demorado em termos de tempo de execução, por testar diversos modelos

In [19]:
submission = test[["PassengerId", "Survived"]]
try:
    f = open("data/results.csv", "w+")
except:
    f = open("data/results.csv", "w+")
f.write(submission.to_csv(index=False))
f.close()

# Por fim, seguem os resultados obtidos pelos testes no kaggle e o link para o YouTube

Vídeo explicativo do Perceptron: https://www.youtube.com/watch?v=agATTwsIzm4

![title](results.png)